In [8]:
import os
import autogen
from dotenv import load_dotenv

from autogen import config_list_from_json
import autogen

load_dotenv()

config_list_file_name = ".config_list"

gemini_config_list = config_list_from_json(config_list_file_name, filter_dict={"tags": ["gemini"]})
gemini_llm_config = {"config_list": gemini_config_list, "timeout": 120}

code_execution_config={
        "last_n_messages": 3,
        "work_dir": "documentation",
        "use_docker": False,
    }

llm_config = {
    "config_list": gemini_llm_config,
    "temperature": 0.2,
    "seed": 42,
}

assistant = autogen.AssistantAgent(
    name="Expert_Summarizer_Agent",
    description="Your task is to understand the semantic and summarizing the given content of a file by yourself, without generating any code",
    code_execution_config=False,
    llm_config=gemini_llm_config,
    system_message="""Please understand and then summarize the purpose of the content of a file within 5-7 lines by yourself."""
)

assistant2 = autogen.AssistantAgent(
    name="Project_Document_Creator_Agent",
    llm_config=gemini_llm_config,
    code_execution_config=code_execution_config,
    system_message="""You are an expert on preparing readme.md content with the help of Expert_Summarizer_Agent.
    Your task is as followings:
    1. run a `find` command in the bash to list the files and save the output on file tree.txt
    2. Here is how you should do it,
        # generate tree.txt by using following exact command from your current directory.
        ```cd ../../ && find . \( -name "*.json" -o -name "*.py" -o -name "*.ipynb" \) -type f -not -path "*/\.*" -exec echo "../../{}" \; > tree.txt && cd - && mv ../../tree.txt .```
    3. Now iterate over all the files listed in the tree.txt and for each file, perform the following steps:
        3.1 - Read the code of the file.
        3.2 - Send the content of the file to Expert_Summarizer_Agent and get the summarization from it.
            - Do not generate code for summarization step, instead do it yourself, (as ab LLM agent)
        3.3 - Append the summary in the file named readme.md as markdown with proper readable entries.
        
        Keep repeating 3.1 and 3.3 until all files have been traversed.

    IMPORTANT: You must do these steps (3.1 ~ 3.4) one by one for each file. For each file you may need to call Expert_Summarizer_Agent.
    Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
    When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
    If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
    If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
    When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
    Reply "TERMINATE" in the end when everything is done."""
)

user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    description="You are a human proxy agent need to get the job done with other assistants",
    human_input_mode="NEVER",
    llm_config=gemini_llm_config,
    code_execution_config=code_execution_config
)

def generate_technical_review():
    groupchat = autogen.GroupChat(
        agents=[user_proxy, assistant, assistant2],
        messages=[],
        max_round=20,
        speaker_selection_method="auto",
        allow_repeat_speaker=True
    )
    
    # Create manager
    manager = autogen.GroupChatManager(
        groupchat=groupchat,
        llm_config=gemini_llm_config
    )

    user_proxy.initiate_chat(
        manager,
        message=f"""Your task is to make sure that readme.md file is generated as per instructions."""
    )

if __name__ == "__main__":
    if not os.path.exists("documentation"):
        os.makedirs("documentation")
    generate_technical_review()

User_Proxy (to chat_manager):

Your task is to make sure that readme.md file is generated as per instructions.

--------------------------------------------------------------------------------

Next speaker: Project_Document_Creator_Agent

Project_Document_Creator_Agent (to chat_manager):

Plan:

1. **Generate `tree.txt`**: Execute the provided bash command to create a file named `tree.txt` containing the paths of all `.json`, `.py`, and `.ipynb` files.

2. **Iterate and Summarize**: Read `tree.txt` line by line. For each line (representing a file path):
    - Open and read the file content.
    - Summarize the content using my abilities as Expert_Summarizer_Agent.
    - Append the filename and summary to `readme.md` in markdown format.

3. **Final Output**: `readme.md` will contain the summarization of all the code files.

---

**Step 1: Generate `tree.txt`**

```bash
cd ../../ && find . \( -name "*.json" -o -name "*.py" -o -name "*.ipynb" \) -type f -not -path "*/\.*" -exec echo "../

In [15]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

config_list = config_list_from_json(env_or_file=config_list_file_name)

code_agent = UserProxyAgent(
    name="CodeAgent",
    system_message="""You are an expert in file operations. Your tasks:
    1. Read file lists from tree.txt
    2. Process files sequentially
    3. Store summaries
    4. Write final README.md
    Only write code for file operations. For summaries, consult the SummarizeAgent.""",
    code_execution_config={"work_dir": "documentation"},
    human_input_mode="NEVER",
    llm_config=gemini_llm_config
)

summarize_agent = AssistantAgent(
    name="SummarizeAgent",
    system_message="""You are a professional summarizer. Provide clear, concise summaries of technical content.
    1. Focus on key functionalities
    2. Highlight technical components
    3. Never write code
    4. Keep summaries under 100 words
    Return only the summary text.""",
    llm_config=gemini_llm_config
)

def generate_readme():
    try:
        with open("documentation/tree.txt", "r") as f:
            file_paths = [line.strip() for line in f if line.strip()]
        summaries = []
        for path in file_paths:
            with open(path, "r") as f:
                content = f.read()
            code_agent.initiate_chat(
                summarize_agent,
                message=f"Summarize this {path.split('/')[-1]} file's purpose and key features:\n\n{content}"
            )
            summary = summarize_agent.last_message()["content"]
            summaries.append(f"## {path}\n\n{summary}\n")
        with open("documentation/README.md", "w") as f:
            f.write("# Project File Summary\n\n")
            f.write("\n".join(summaries))
    except Exception as e:
        print(e)

    return "README.md generated with file summaries"

code_agent.execute_code_blocks([('python', 'generate_readme()')])
print("Check documentation/README.md for the generated summary file!")


>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
Check documentation/README.md for the generated summary file!
